In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import math
from IPython.display import display

In [2]:
esun_occ_id_to_str = {
    1: "營造／礦砂業",
    2: "製造業",
    3: "水電燃氣業",
    4: "批發／零售／貿易商",
    5: "旅宿／餐飲",
    6: "運輸倉儲",
    7: "農／林／漁／牧",
    8: "金融保險",
    9: "不動產／租賃",
    10: "軍／警／消",
    11: "公務人員",
    12: "律師／會計師／地政士",
    13: "醫藥服務",
    14: "休閒服務",
    15: "其他／家管",
    16: "自由業",
    17: "投資／自營商",
    18: "媒體文教",
    19: "學生",
    20: "學校教師（含行政人員）",
    21: "資訊科技",
    22: "公證人或記帳士",
    23: "國防工業",
    24: "投資或稅務顧問公司",
    25: "不動產仲介／代銷商",
    26: "大宗物資貿易商",
    27: "博弈業（網路／實體）",
    28: "八大特種行業",
    29: "宗教、慈善、基金會",
    30: "銀樓、珠寶商",
    31: "藝術品或古董買賣商",
    32: "當鋪",
    33: "實體或虛擬貨幣兌換所"
}

gov_occ_str_to_id = {
    '礦業及土石採取業' : 0,
    '製造業' : 1,
    '電力及燃氣供應業' : 2,
    '用水供應及污染整治業' : 3,
    '營建工程業' : 4,
    '批發及零售業' : 5,
    '運輸及倉儲業' : 6,
    '住宿及餐飲業' : 7,
    '出版、影音製作、傳播及資通訊服務業' : 8,
    '金融及保險業' : 9,
    '不動產業' : 10,
    '專業、科學及技術服務業' : 11,
    '支援服務業' : 12,
    '教育業' : 13,
    '醫療保健及社會工作服務業' : 14,
    '藝術、娛樂及休閒服務業' : 15,
    '其他服務業' : 16
}

gov_occ_list = ['礦業及土石採取業', '製造業', '電力及燃氣供應業', '用水供應及污染整治業', '營建工程業', '批發及零售業',
       '運輸及倉儲業', '住宿及餐飲業', '出版、影音製作、傳播及資通訊服務業', '金融及保險業', '不動產業',
       '專業、科學及技術服務業', '支援服務業', '教育業', '醫療保健及社會工作服務業', '藝術、娛樂及休閒服務業',
       '其他服務業']

lead_job_list = [
    '主管及監督人員',
    '專業人員',
    '技術員及助理專業人員',
    '事務支援人員',
    '服務及銷售工作人員',
    '技藝、機械設備操作及組裝人員',
    '基層技術工及勞力工',
]

lead_job_str_to_id = {
    '主管及監督人員' : 0,
    '專業人員' : 1,
    '技術員及助理專業人員' : 2,
    '事務支援人員' : 3,
    '服務及銷售工作人員' : 4,
    '技藝、機械設備操作及組裝人員' : 5,
    '基層技術工及勞力工' : 6 
}

In [3]:
df_gov_occ = pd.read_csv('data/occ.csv')

def esun_occ_id_to_gov_occ_id(esun_occ_id):
    
    if esun_occ_id == 99 or math.isnan(esun_occ_id) or esun_occ_id is None:
        return 16 # '其他服務業'
    
    gov_str = df_gov_occ['gov_occ'][esun_occ_id - 1]
    if gov_str[0] != '*':
        gov_occ_id = gov_occ_str_to_id[gov_str]
    elif gov_str[1] == '*':
        return -int(gov_str[2:])
    else:
        return -int(gov_str[1:])
    return gov_occ_id

In [4]:
df_industry = pd.read_excel('./data/psdnquery1-9.xlsx')
df_industry = df_industry.iloc[:, 1:].drop([0, 1, 2]).reset_index(drop=True)
df_industry.columns = gov_occ_list
df_industry

,礦業及土石採取業,製造業,電力及燃氣供應業,用水供應及污染整治業,營建工程業,批發及零售業,運輸及倉儲業,住宿及餐飲業,出版、影音製作、傳播及資通訊服務業,金融及保險業,不動產業,專業、科學及技術服務業,支援服務業,教育業,醫療保健及社會工作服務業,藝術、娛樂及休閒服務業,其他服務業
0,"205,785","129,663","342,430","150,543","82,958","97,337","121,873","55,708","106,808","139,925","78,081","105,969","61,535","49,354","114,097","75,026","58,622"
1,"208,883","99,948","223,198","105,235","67,752","72,007","74,113","52,045","74,207","84,057","61,911","70,712","48,336","29,817","69,461","38,376","49,275"
2,"183,072","66,743","122,896","80,283","49,014","62,228","80,590","46,527","69,515","69,188","52,993","47,218","39,806","36,168","38,792","42,812","38,582"
3,"74,574","40,641","167,448","71,126","35,122","37,317","82,548","32,202","46,935","56,071","38,615","35,265","36,037","25,579","32,095","32,813","31,326"
4,"220,135","43,633","168,871","50,737","33,323","37,063","43,885","28,996","52,050","72,766","44,776","34,680","32,764","29,149","29,695","32,839","29,769"
5,"61,953","40,217","213,451","57,609","41,399","41,297","48,156","36,130","92,417","70,247","50,423","38,398","35,985","24,511","38,812","39,052","33,276"
6,"25,344","30,683","41,453","28,232","29,329","30,137","42,710","25,774","21,653","43,240","29,758","25,859","28,827","21,038","28,431","28,632","27,793"


In [5]:
foo = df_industry.applymap(lambda x: x.replace(",", "")).astype(int)
x = foo.mean(axis=1)
m = x.mean()
s = x.std()
list((x - m)/s)


[1.8282004788567674,
 0.6560752369233088,
 0.0062653633495642356,
 -0.5315773430784837,
 -0.2968502431305844,
 -0.34361245855107936,
 -1.3185010343694934]

In [6]:
def to_class(x):
    '''將quota分class'''
    
    '''
    0~10萬
    10~30萬(不含10萬)
    30~50萬(不含30萬)
    50~100萬(不含50萬)
    '''
    if x < 1E5:
        return 0
    if 1E5 <= x and x < 3E5:
        return 1
    if 3E5 <= x and x < 5E5:
        return 2
    else:
        return 3

def f(a, b):
    '''取quota_now與electric_now的min'''
    if math.isnan(b):
        return a
    else:
        return min(a, b)

In [25]:
# 玉山資料
df_esun = pd.read_csv('./data/job.csv')
df_esun['job_title'] = df_esun['job_title'].apply(lambda x: str(x).strip())
df_esun = df_esun[df_esun['quota_now']<=1e6] # consider only quota in [0, 1E6], endpoints included
df_esun['y_num'] = df_esun[['quota_now', 'quota_now_elec']].apply(lambda x: f(*x), axis=1)
df_esun['y_cat'] = df_esun['y_num'].apply(lambda x: to_class(x))
df_esun = df_esun.drop(['quota_origin_elec', 'quota_now_elec', 'quota_origin', 'quota_now'], axis=1)
# df_esun = df_esun[df_esun['occupation'] <= 33]
# df_esun = df_esun.dropna()
display(df_esun.head())

,job_title,occupation,y_num,y_cat
0,維護工程師,21.0,500000.0,3
1,攝影記者,18.0,300000.0,2
3,工程師,2.0,300000.0,2
4,高級工程師,2.0,300000.0,2
5,業務,21.0,1000000.0,3


In [26]:
df_esun['occupation'].value_counts()

2.0     20819
15.0    13784
4.0     12662
21.0    11553
19.0    10217
13.0     9148
5.0      7343
20.0     5544
11.0     5276
14.0     5176
8.0      4316
16.0     3780
1.0      3437
10.0     3366
6.0      3107
18.0     3046
9.0      1041
3.0      1020
12.0      994
17.0      931
7.0       730
29.0      703
25.0      342
22.0      145
24.0      139
23.0      113
28.0       95
27.0       77
30.0       70
26.0       58
31.0       53
33.0       17
32.0       15
99.0        4
Name: occupation, dtype: int64

In [27]:
df_jobs = pd.read_excel('./data/職等分類依據.xlsx')
df_jobs.head()

,各業受僱員工(人數)(107年7月)(單位：人)
0,(100000)主管及監督人員
1,(112000)高階主管(總經理及總執行長)
2,(120090)中階主管(經理)
3,(312190)監督人員(含現場及辦公室主任、科/股長)
4,(200000)專業人員


In [45]:
def get_job_list(raw_job_data_path = './data/職等分類依據.xlsx'):
    '''得到job_list與lead_list'''
    df_jobs = pd.read_excel(raw_job_data_path)
    df_jobs.head()

    raw_job_list = df_jobs['各業受僱員工(人數)(107年7月)(單位：人)'].values

    job_list = []
    lead_list = []

    for idx, x in enumerate(raw_job_list):
        if x[0]=='(':
            lead_list.append(idx)
        x = str(x)
        x = re.sub(r'[^\w]', '', x)
        x = re.compile(u'[\u4E00-\u9FA5|\s]').findall(x)
        x = "".join(x)
        if x:
            job_list.append(x)
        else:
            job_list.append('無')
            
    # print(job_list[:10])
    # print(lead_list)
    return job_list, lead_list

def get_lead_job_idx(sub_job_id: int, lead_list: list[int]) -> int: 
    '''返回某職業id對應的職業類別id'''
    for i in range(len(lead_list)-1):
        if lead_list[i] <= sub_job_id and sub_job_id < lead_list[i+1]:
            return lead_list[i]
    return lead_list[-1]

def find_most_similar_job(job_name: str, job_list: list[str]) -> int:
    '''給定職業名回傳最相似的職業id'''
    score_list = []
    for x in job_list:
        score_list.append(nltk.edit_distance(list(x), list(job_name)))
    return np.argsort(score_list)[0]

def get_lead_job_name(job_name: str, job_list: list[str], lead_list: list[int]) -> str:
    '''給定職業名回傳最相似的職業類別名'''
    return job_list[get_lead_job_idx(find_most_similar_job(job_name, job_list), lead_list)]


In [83]:
salary_backup = pd.read_excel('data/Year19.xls', skiprows=11)
salary_backup

,全體所得收入者：\nAll income recipients,557634,1.81,571518,2.49,454169,2.22,468263,3.1
0,按行業別分：Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1.農林漁牧業\n Agriculture,Forestry,Fishing and\n...",454387.0,5.04,452930.0,-0.32,356370.0,6.11,363241.0,1.93
2,2.礦業、土石採取業及製造業\n Mining and Quarrying and\n ...,617600.0,2.48,651913.0,5.56,488972.0,2.56,516615.0,5.65
3,"3.電力、燃氣、用水供應及汙染\n 整治業\n Electricity,Gas,Wa...",747450.0,9.91,715330.0,-4.30,650881.0,5.83,621763.0,-4.47
4,4.營造業\n Construction,613442.0,4.49,608293.0,-0.84,516875.0,3.83,521073.0,0.81
5,5.批發及零售業\n Wholesale and Retail Trade,594944.0,3.23,618178.0,3.91,472667.0,3.82,495771.0,4.89
6,6.住宿及餐飲業\n Accommodation and Food Services,485616.0,-0.23,510027.0,5.03,400021.0,2.07,431396.0,7.84
7,7.運輸、倉儲、資訊及通訊傳播業\n Transportation and Stora...,656600.0,0.39,676089.0,2.97,576070.0,1.40,597730.0,3.76
8,8.金融、保險及不動產業\n Finance and Insurance and\n ...,792769.0,4.97,846688.0,6.80,673835.0,6.33,688230.0,2.14
9,"9.專業、科學及技術服務業\n Professional, Scientific and...",726289.0,-2.33,745248.0,2.61,594202.0,2.05,595156.0,0.16


In [147]:
def get_salary(job_name_str, job_list, lead_list, esun_occ_id, salary_backup):
    rel_val = [   
        1.8282004788567674,
        0.6560752369233088,
        0.0062653633495642356,
        -0.5315773430784837,
        -0.2968502431305844,
        -0.34361245855107936,
        -1.3185010343694934
    ]
    lead_job_id = lead_job_str_to_id[get_lead_job_name(job_name_str, job_list, lead_list)]
    occ_id = esun_occ_id_to_gov_occ_id(esun_occ_id)
    if occ_id >= 0:
        return int(df_industry.iloc[lead_job_id, occ_id].replace(',', ""))
    else:
        if occ_id in {-1, -15}:
            return int(salary_backup.iloc[-occ_id, 1] * (1 + rel_val[lead_job_id])) // 12
        else:
            return int(salary_backup.iloc[-occ_id, 1]) // 12

In [148]:
job_list, lead_list = get_job_list()
get_lead_job_name('老師', job_list, lead_list)

'服務及銷售工作人員'

In [149]:
get_salary('老師', job_list, lead_list, 7, salary_backup)

26625

# 嘗試中文詞向量模型

In [25]:
# from text2vec import Similarity
# sim_text2vec = Similarity()
# score = sim_text2vec.get_score('工程師', '電腦工程師')
# print(score)

from similarities import Similarity
sim_model = Similarity()

2022-05-28 12:28:21.472 | DEBUG    | text2vec.sentence_model:__init__:74 - Use device: cuda


In [27]:
r = sim_model.similarity('水', '礦泉水')
print(f"similarity score: {float(r)}")  # similarity score: 0.855146050453186

similarity score: 0.6244003772735596
